# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pypokerengine.players import BasePokerPlayer
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate
import inspect
from pypokerengine.engine.hand_evaluator import HandEvaluator
from pypokerengine.engine.card import Card


# Honest Player

In [ ]:
NB_SIMULATION = 100
class HonestPlayer(BasePokerPlayer):
    def __init__(self, raise_wr = 0.6, call_wr = 0.3):
        super().__init__() 
        self.raise_wr = raise_wr
        self.call_wr = call_wr
        
    def declare_action(self, valid_actions, hole_card, round_state):
        curframe = inspect.currentframe()
        calframe = inspect.getouterframes(curframe, 2)
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(
                nb_simulation=NB_SIMULATION,
                nb_player=self.nb_player,
                hole_card=gen_cards(hole_card),
                community_card=gen_cards(community_card)
                )
        if win_rate >= self.raise_wr:
            #print('Raising)', win_rate)
            action = valid_actions[2]  # fetch bet action info
            valid_amounts = action['amount']
            raise_amount =  (valid_amounts['max'] - valid_amounts['min'])/ 5.0 + valid_amounts['min']
            return valid_actions[2]['action'], raise_amount
        elif win_rate >= self.call_wr:
            #print('Calling')
            action = valid_actions[1]  # fetch CALL action info
        else:
            call_amount = valid_actions[1]['amount']
            if(call_amount == 0):
                #print('Calling')
                return valid_actions[1]['action'], valid_actions[1]['amount']
            #print('Folding')
            action = valid_actions[0]  # fetch FOLD action info
        return action['action'], action['amount']

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

# My Player

In [ ]:
def iterate_actions(hand_data):
    """Iterates over all actions in the 'action_histories' of a given hand.
    
    Args:
    hand_data: A dictionary representing a hand of poker.
    
    Yields:
    A tuple containing the action type, the amount, the player's UUID, and the round.
    """
    if(hand_data):
        for round_name, actions in hand_data['action_histories'].items():
            for action in actions:
                yield (action['action'], action.get('amount'), action['uuid'], round_name)

class BijPlayer(BasePokerPlayer):
    def __init__(self):
        super().__init__() 
        self.player_histories = {}
        self.player_tendencies = {}
        self.hand_evaluator = HandEvaluator()
        self.INVERTED_HAND_STRENGTH_MAP = {value: key for key, value in self.hand_evaluator.HAND_STRENGTH_MAP.items()}

        self.player_showdown_score_histories = {}
        self.player_showdown_score_metrics = {}

        self.player_states = []

    def make_score(self, ri):
        hand_strength = self.INVERTED_HAND_STRENGTH_MAP[ri['hand']['strength']]
        hand_high = ri['hand']['high']
        hand_low = ri['hand']['low']
        hole_high = ri['hole']['high']
        hole_low = ri['hole']['low']
        return (hand_strength << 8) | (hand_high << 12) | (hand_low << 8) | (hole_high << 4) | hole_low

    # def dfs_simulation(self, next_player, seats):
    #     # Get the valid actions for the player

    #     # Get the value of each action

    #     # Get the probability of each action

    #     # Calculate the EV from each action as a weighted sum 
    #     ev_sum = prob_fold * value_fold + prob_call * value_call + prob_raise * value_raise
        
    def declare_action(self, valid_actions, hole_card, round_state):
        next_player = round_state['next_player']

        player_costs = {}
        # Sum the total cost so far for each player
        #for action in round_state['action_histories']:
        #    player_costs[action['uuid']] += action['amount']

        participating_players = [player for player in round_state['seats'] if(player['state'] == 'participating')]
                
            
            
        
        # Determine the round state.
        # print('-------------------start--------------')
        # print('round_state: ', round_state.keys())
        # print('round_state: ', round_state['action_histories'])
        # print('round_state: ', round_state['seats'])
        # print('-------------------end--------------')
        # Calculate the EV of each action: 

        # Expected value of the fold action.
        ev_fold = 0
        # Expected value of the call action.
        #ev_call = calc_ev('call')
        # test
        return valid_actions[1]['action'], valid_actions[1]['amount']

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']
        

    def receive_round_start_message(self, round_count, hole_card, seats):
        print('On round: ', round_count)
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        self.player_states += [round_state['seats'][round_state['next_player']]['state']]
        #pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        #print('Round state: ', round_state)
        
        
        print('\n\n')
        showdown_players_uuids = [player['uuid'] for player in round_state['seats'] if(player['state'] == 'participating')]
        folded_players_uuids = [player['uuid'] for player in round_state['seats'] if(player['state'] == 'folded')]

        players_showdown_states = {player['uuid']: player['state'] for player in round_state['seats']}

        print('\nhand info: ', hand_info)
        print('showdown player: ', showdown_players_uuids)

        # Get cost for each player
        player_costs = {}
        for street, player_actions in round_state['action_histories'].items():
            for action in player_actions:
                uuid = action['uuid']
                try:
                    amount = action['amount']
                except KeyError:
                    amount = 0
                try:
                    player_costs[uuid] += amount
                except KeyError:
                    player_costs[uuid] = amount

        # NOTE THIS DOES NOT INCLUDE SIDE POTS
        pot_size = round_state['pot']['main']['amount']
        
        # Record player showdown states (score, cost, pot)
        showdown_player_hands = {player['uuid']: player['hand'] for player in hand_info}
        for player_uuid, showdown_state in players_showdown_states.items():
            if(player_uuid in showdown_player_hands):
                hand = showdown_player_hands[player_uuid]
                score = self.make_score(hand)
            else:
                score = None

            try:
                cost = player_costs[player_uuid]
            except KeyError:
                cost = 0
            showdown_state = {'state': players_showdown_states[player_uuid], 'score': score, 'pot': pot_size, 'cost': cost}
            try:
                self.player_showdown_score_histories[player_uuid] += [showdown_state]
            except KeyError:
                self.player_showdown_score_histories[player_uuid] = [showdown_state]
            #print(player_uuid, 'Showdown state: ', showdown_state)
    
        print('Showdown histories: ', self.player_showdown_score_histories)
        
        #print('\n')
            

# Run Poker Game

In [ ]:
from collections import Counter

results_dict = {'p1': [], 'p2': [], 'p3': [], 'p4': []}
initial_stack = 1000
for i in range(1):
    config = setup_config(max_round=1000, initial_stack=initial_stack, small_blind_amount=5)
    config.register_player(name="p1", algorithm=HonestPlayer())
    config.register_player(name="p2", algorithm=HonestPlayer(raise_wr = 0.9, call_wr=0.7))
    config.register_player(name="p3", algorithm=HonestPlayer(raise_wr = 0.3, call_wr=0.1))
    bijPlayer = BijPlayer()
    config.register_player(name="p4", algorithm=bijPlayer)
    game_result = start_poker(config, verbose=0)
    #print(i, game_result)
    players = game_result['players']
    for player in players:
        results_dict[player['name']] += [player['stack'] - initial_stack]
        
        

print(results_dict)
print(Counter(bijPlayer.player_states))


# Plot Results

In [ ]:

def plot_running_sums(data):
  """Plots the running sums of a dictionary of arrays.

  Args:
    data: A dictionary where the keys are labels and the values are arrays of numbers.
  """

  for label, values in data.items():
    running_sum = np.cumsum(values)
    plt.plot(running_sum, label=label)

  plt.xlabel("Index")
  plt.ylabel("Running Sum")
  plt.legend()
  plt.show()

plot_running_sums(results_dict)